# Creating a Spark SQL View from a Mordor Dataset

* **Author**: Jose Rodriguez (@Cyb3rPandah)
* **Project**: Infosec Jupyter Book
* **Public Organization**: [Open Threat Research](https://github.com/OTRF)
* **License**: [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/)
* **Reference**: https://mordordatasets.com/introduction.html

## Extracting Mordor JSON File

### Get compressed Zip file

We could use the **wget** command and the **-O** (output document file) option to save the file in a **datasets** folder. However, to keep it simple for this workshop, we already downloaded the dataset for you.

! wget https://raw.githubusercontent.com/hunters-forge/mordor/master/datasets/large/apt29/day1/apt29_evals_day1_manual.zip  -O datasets/apt29_evals_day1_manual.zip

### Extract JSON file

We are using the **unzip** command and **-o** (Overwrite) and **-d** (different directory) options to save the file in **datasets** folder

In [1]:
! unzip -o ../datasets/apt29_evals_day1_manual.zip -d ../datasets/

Archive:  ../datasets/apt29_evals_day1_manual.zip
  inflating: ../datasets/apt29_evals_day1_manual_2020-05-01225525.json  


We will store the **path** of the json file in a variable to facilitate our code

In [2]:
apt29Json = '../datasets/apt29_evals_day1_manual_2020-05-01225525.json'

## Creating a SQL View

### Create Spark session

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Mordor") \
    .config("spark.sql.caseSensitive","True") \
    .getOrCreate()

### Read JSON file

In [4]:
%%time
apt29Df = spark.read.json(apt29Json)

CPU times: user 4.44 ms, sys: 2.46 ms, total: 6.89 ms
Wall time: 27.9 s


In [5]:
apt29Df.show(n = 1, vertical = True)

-RECORD 0-----------------------------------------------
 @timestamp                      | 2020-05-01 22:55:... 
 @version                        | 1                    
 AccessList                      | null                 
 AccessMask                      | null                 
 AccessReason                    | null                 
 AccountName                     | SYSTEM               
 AccountType                     | User                 
 Action                          | null                 
 Active                          | null                 
 ActiveProfile                   | null                 
 ActivityID                      | null                 
 AdapterName                     | null                 
 AdapterSuffixName               | null                 
 AdditionalInfo                  | null                 
 AdditionalInfo2                 | null                 
 AdvancedOptions                 | null                 
 AlgorithmName                 

### Expose the dataframe as a SQL view

In [6]:
apt29Df.createOrReplaceTempView('apt29')

## Analyzing the APT29 dataset

Filtering on **Sysmon event 1: Process Creation**

In [7]:
sysmon1 = spark.sql(
'''
SELECT Image, ProcessId, ProcessGuid
FROM apt29
WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
    AND EventID = 1
''')
sysmon1.show(n = 5, truncate = False)

+---------------------------------------------------------+---------+--------------------------------------+
|Image                                                    |ProcessId|ProcessGuid                           |
+---------------------------------------------------------+---------+--------------------------------------+
|C:\ProgramData\victim\â€®cod.3aka3.scr                   |8524     |{47ab858c-e13c-5eac-a903-000000000400}|
|C:\Windows\System32\conhost.exe                          |5156     |{47ab858c-e144-5eac-aa03-000000000400}|
|C:\Windows\System32\cmd.exe                              |2772     |{47ab858c-e144-5eac-ab03-000000000400}|
|C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|5944     |{47ab858c-e14e-5eac-ac03-000000000400}|
|C:\Windows\System32\SearchProtocolHost.exe               |4152     |{47ab858c-e17d-5eac-ad03-000000000400}|
+---------------------------------------------------------+---------+--------------------------------------+
only showing top 5 

## Thank you! I hope you enjoyed it!